In [1]:
## This contains a pure IK based troting motion plan
## Author : Avadesh Meduri
## Date : 26/02/2021

import time
import numpy as np
import pinocchio as pin
import crocoddyl

from robot_properties_solo.config import Solo12Config
from py_biconvex_mpc.ik.inverse_kinematics import InverseKinematics
from py_biconvex_mpc.ik_utils.gait_generator import GaitGenerator


solo import 


In [2]:
dt = 1e-2
T = 2.0

robot = Solo12Config.buildRobotWrapper()
q0 = np.array(Solo12Config.initial_configuration)
x0 = np.concatenate([q0, pin.utils.zero(robot.model.nv)])

pin.forwardKinematics(robot.model, robot.data, q0, pin.utils.zero(robot.model.nv))
pin.updateFramePlacements(robot.model, robot.data)

fl_loc = robot.data.oMf[robot.model.getFrameId("FL_FOOT")].translation
fr_loc = robot.data.oMf[robot.model.getFrameId("FR_FOOT")].translation
hl_loc = robot.data.oMf[robot.model.getFrameId("HL_FOOT")].translation
hr_loc = robot.data.oMf[robot.model.getFrameId("HR_FOOT")].translation


In [3]:
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=True)
viz.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [4]:
sl = np.array([0.1, 0.0, 0])
mass = pin.computeTotalMass(robot.model, robot.data)
com_loc = pin.centerOfMass(robot.model, robot.data, q0)
ctraj = np.zeros((int(T/dt), 6))
ctraj[:,0] = 3*mass*sl[0]/T

ori_traj = np.tile(np.eye(3), ((int(T/dt), 1, 1))) 

In [5]:
gg = GaitGenerator(robot, T, dt)

# gg.create_centroidal_task(ctraj, 0, 2.0, "cmom_task", 1e5)

gg.create_swing_foot_task(fl_loc, fl_loc + sl, 0.0, 0.5, 0.1, "FL_FOOT", "FL_ftc1", 1e4)
gg.create_swing_foot_task(hr_loc, hr_loc + sl, 0.0, 0.5, 0.1, "HR_FOOT", "HR_ftc1", 1e4)
gg.create_contact_task(fr_loc, 0.0, 0.5, "FR_FOOT", "FR_ctc1", 1e3)
gg.create_contact_task(hl_loc, 0.0, 0.5, "HL_FOOT", "HL_ctc1", 1e3)

gg.create_swing_foot_task(fr_loc, fr_loc + 2*sl, 0.5, 1.0, 0.1, "FR_FOOT", "FR_ftc1", 1e4)
gg.create_swing_foot_task(hl_loc, hl_loc + 2*sl, 0.5, 1.0, 0.1, "HL_FOOT", "HL_ftc1", 1e4)
gg.create_contact_task(fl_loc + sl, 0.5, 1.0, "FL_FOOT", "FL_ctc1", 1e3)
gg.create_contact_task(hr_loc + sl, 0.5, 1.0, "HR_FOOT", "HR_ctc1", 1e3)

gg.create_swing_foot_task(fl_loc + sl, fl_loc + 3*sl, 1.0, 1.5, 0.1, "FL_FOOT", "FL_ftc1", 1e4)
gg.create_swing_foot_task(hr_loc + sl, hr_loc + 3*sl, 1.0, 1.5, 0.1, "HR_FOOT", "HR_ftc1", 1e4)
gg.create_contact_task(fr_loc + 2*sl, 1.0, 1.5, "FR_FOOT", "FR_ctc2", 1e3)
gg.create_contact_task(hl_loc + 2*sl, 1.0, 1.5, "HL_FOOT", "HL_ctc2", 1e3)

gg.create_swing_foot_task(fr_loc + 2*sl, fr_loc + 4*sl, 1.5, 2.0, 0.1, "FR_FOOT", "FR_ftc1", 1e4)
gg.create_swing_foot_task(hl_loc + 2*sl, hl_loc + 4*sl, 1.5, 2.0, 0.1, "HL_FOOT", "HL_ftc1", 1e4)
gg.create_contact_task(fl_loc + 3*sl, 1.5, 2.0, "FL_FOOT", "FL_vtc2", 1e3)
gg.create_contact_task(hr_loc + 3*sl, 1.5, 2.0, "HR_FOOT", "HR_vtc2", 1e3)

# gg.ik.add_orientation_tracking_task(robot.model.getFrameId("root_joint"), 0, 2.0, ori_traj, 1e5, "base_ori")
# comTrack = crocoddyl.CostModelCoMPosition(gg.ik.state, 4*sl + com_loc)
# gg.ik.terminalCostModel.addCost("comtask", comTrack, 1e+3)

xs = gg.optimize(x0, wt_xreg=5e-4)


In [6]:
for i in range(len(xs)):
    time.sleep(0.01)
    viz.display(xs[i][:robot.model.nq])

In [7]:
for i in range(len(xs)):
    print(xs[i][3:7])

[0. 0. 0. 1.]
[ 2.29411185e-05 -5.48623102e-06  1.75178095e-06  1.00000000e+00]
[1.50485127e-05 5.60861775e-05 3.62379960e-06 9.99999998e-01]
[-2.78178499e-05  1.44988322e-04  5.63545880e-06  9.99999989e-01]
[-7.61287172e-05  2.19743653e-04  7.77017395e-06  9.99999973e-01]
[-1.06319634e-04  2.59329134e-04  9.98513444e-06  9.99999961e-01]
[-1.11363649e-04  2.62980134e-04  1.22424786e-05  9.99999959e-01]
[-9.57729097e-05  2.40731832e-04  1.45193915e-05  9.99999966e-01]
[-6.79704426e-05  2.04692606e-04  1.68040742e-05  9.99999977e-01]
[-3.53776166e-05  1.64370868e-04  1.90904862e-05  9.99999986e-01]
[-2.63827325e-06  1.25479677e-04  2.13757179e-05  9.99999992e-01]
[2.82232393e-05 9.05997676e-05 2.36589276e-05 9.99999995e-01]
[5.68410524e-05 6.04240245e-05 2.59406204e-05 9.99999996e-01]
[8.33677583e-05 3.49323702e-05 2.82218899e-05 9.99999996e-01]
[1.07383434e-04 1.42824627e-05 3.05036251e-05 9.99999994e-01]
[ 1.26918844e-04 -6.37949782e-07  3.27857265e-05  9.99999991e-01]
[ 1.37707857e-04